# reactionrnn Demo

by [Max Woolf](http://minimaxir.com)

*Max's open-source projects are supported by his [Patreon](https://www.patreon.com/minimaxir). If you found this project helpful, any monetary contributions to the Patreon are appreciated and will be put to good creative use.*

## Intro

reactionrnn is a Python module on top of Keras/TensorFlow which can easily predict the proportionate reactions (love, wow, haha, sad, angry) to a given text using a pretrained recurrent neural network:

In [1]:
from reactionrnn import reactionrnn

react = reactionrnn()

Using TensorFlow backend.


The `reactionrnn` initializer builds a normal Keras model using pretrained weights, and can be accessed using the `.model` attribute:

In [2]:
react.model.summary()
react.model.get_layer('rnn').get_weights()[0]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 140)               0         
_________________________________________________________________
embedding (Embedding)        (None, 140, 100)          40100     
_________________________________________________________________
rnn (GRU)                    (None, 256)               274176    
_________________________________________________________________
output (Dense)               (None, 5)                 1285      
Total params: 315,561
Trainable params: 315,561
Non-trainable params: 0
_________________________________________________________________


array([[ 0.61926103,  0.05955176, -0.19037187, ...,  0.04486968,
        -0.1982885 , -0.13359   ],
       [ 0.2541163 ,  0.82486463, -0.59637159, ...,  0.13716327,
        -0.17285788, -0.24066631],
       [ 0.39044452,  0.26126146, -0.30949306, ..., -0.16419689,
         0.30253917, -0.06857044],
       ..., 
       [-0.24404441, -0.31984991,  0.02359354, ..., -0.07383229,
         0.31076583, -0.13405246],
       [-0.31463742, -0.20329736, -0.4156228 , ...,  0.20338874,
         0.00286744,  0.08513979],
       [-0.8035174 , -0.28070208,  0.35372871, ...,  0.13372001,
        -0.0451884 , -0.05612914]], dtype=float32)

## Predict Reaction to Single Text

The `predict` function returns an `OrderedDict` of the five predicted reactions to the given text, sorted from strongest to weakest.

In [3]:
prediction = react.predict("Happy Mother's Day from the Chicago Cubs!")
print(prediction)

OrderedDict([('love', 0.9765), ('wow', 0.0235), ('haha', 0.0), ('sad', 0.0), ('angry', 0.0)])


In [4]:
print(react.predict("This is scary AF!😱😱"))
print(react.predict("When the soup is too hot 😂😂😂"))
print(react.predict("He was only 41."))
print(react.predict("Everyone loves autoplaying videos!"))

OrderedDict([('wow', 0.9109), ('sad', 0.0891), ('love', 0.0), ('haha', 0.0), ('angry', 0.0)])
OrderedDict([('haha', 0.8568), ('love', 0.1376), ('wow', 0.0056), ('sad', 0.0), ('angry', 0.0)])
OrderedDict([('sad', 1.0), ('love', 0.0), ('wow', 0.0), ('haha', 0.0), ('angry', 0.0)])
OrderedDict([('angry', 0.8667), ('wow', 0.1333), ('love', 0.0), ('haha', 0.0), ('sad', 0.0)])


Like a normal `dict`, you can access the value by key.

In [5]:
prediction['wow']

0.0235

It plays nice with JSON and Python's `json` module, if you want to use reactionrnn for an API.

In [6]:
import json
print(json.dumps(prediction, indent=4))

{
    "love": 0.9765,
    "wow": 0.0235,
    "haha": 0.0,
    "sad": 0.0,
    "angry": 0.0
}


If you just want the strongest predicted label for a given text(s), you can access that with `predict_label`, which returns a `list` containing a reaction for each input. (good for classification)

In [7]:
react.predict_label("Happy Mother's Day from the Chicago Cubs!")

['love']

In [8]:
react.predict_label(["He was only 41.", "Everyone loves autoplaying videos!"])

['sad', 'angry']

## Predict Reaction to Multiple Texts

If you provide a `list` of texts to `predict()`, the return value is a numpy array of (`n`, 5), where `n` is the number of texts. (columns are in the order of `(love, wow, haha, sad, angry)`)

In [9]:
texts = ['Never gonna give you up, never gonna let you down',
            'Never gonna run around and desert you',
            'Never gonna make you cry, never gonna say goodbye',
            'Never gonna tell a lie and hurt you']

react.predict(texts)

array([[ 0.08376055,  0.        ,  0.91623938,  0.        ,  0.        ],
       [ 0.09415939,  0.        ,  0.90584058,  0.        ,  0.        ],
       [ 0.19280881,  0.1212144 ,  0.38475046,  0.30122641,  0.        ],
       [ 0.07649462,  0.        ,  0.92350537,  0.        ,  0.        ]], dtype=float32)

## Encode Text as Vector

You can also encode text as a 256D vector, which unlike those produced by word2vec/doc2vec/fasttext, incorporates the information provided by modern grammar including punctuation and emoji.

In [10]:
encoding = react.encode("DYING. 😄")
print(encoding[0][0:10])
print(encoding.shape)

[ 0.0411452   0.87985831  0.31406021  0.50111514  0.51543093  0.12899202
 -0.23730506  0.07143005  0.20887181  0.95970839]
(1, 256)


You can also encode multiple texts as vectors.

In [11]:
encoding = react.encode(texts)
print(encoding[0:10])
print(encoding.shape)

[[-0.65013576 -0.11578681 -0.32001457 ..., -0.21620744  0.40326133
   0.14626786]
 [ 0.00691345  0.02702127  0.0255055  ..., -0.54953575 -0.22651449
   0.02594155]
 [ 0.38269144 -0.02336972  0.21063511 ...,  0.25455818 -0.36290079
   0.13700774]
 [ 0.14133058  0.02101863 -0.18063194 ..., -0.22906038 -0.57762033
  -0.18347938]]
(4, 256)


I strongly recommend using PCA to both reduce the high dimensionality of the text and align the given encoded texts in context of each other.

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca.fit(encoding)
print(pca.transform(encoding))

[[-1.14232777  3.72827529]
 [-1.65552238 -1.5594023 ]
 [ 4.58220842 -0.34441789]
 [-1.78435827 -1.8244551 ]]


In [13]:
print(pca.explained_variance_ratio_)
print(sum(pca.explained_variance_ratio_))

[ 0.52853034  0.37035823]
0.898888575824


These 2 components explain about 90% of the variation in the text; something which would hard to achieve without overfitting on only 4 text documents!

# LICENSE

MIT License

Copyright (c) 2017 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.